In [1]:
import pandas as pd
import matplotlib.pylab as plt

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
angle_details = pd.read_csv('../data/openbabel/angle_details.csv')
atom_details = pd.read_csv('../data/openbabel/atom_details.csv')
bond_details = pd.read_csv('../data/openbabel/bond_details.csv')
closest = pd.read_csv('../data/openbabel/closest.csv')
distances = pd.read_csv('../data/openbabel/distances.csv')
molecule_details = pd.read_csv('../data/openbabel/molecule_details.csv')
torsion_details = pd.read_csv('../data/openbabel/torsion_details.csv')

In [4]:
tt = pd.concat([train, test], axis=0, sort=False)

In [5]:
tt.shape

(7163689, 6)

In [6]:
tt.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [7]:
tt = pd.merge(tt,
         angle_details.groupby(['molecule_name','left_atom_idx','middle_atom_idx']).mean().reset_index(drop=False),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','middle_atom_idx'],
         how='left') \
    .drop(['left_atom_idx','middle_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'angle':'left_middle_average_angle'})

In [8]:
tt.shape

(7163689, 7)

In [9]:
tt = pd.merge(tt,
         angle_details.groupby(['molecule_name','right_atom_idx','middle_atom_idx']).mean().reset_index(drop=False),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','right_atom_idx','middle_atom_idx'],
         how='left') \
    .drop(['left_atom_idx','middle_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'angle':'right_middle_average_angle'})

In [10]:
tt.shape

(7163689, 8)

In [11]:
tt = pd.merge(tt,
         distances,
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','right_atom_idx'],
         how='left')\
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

In [12]:
tt.shape

(7163689, 10)

In [13]:
closest.head()

,molecule_name,atom_index,closest,2nd_closest,3rd_closest,4th_closest,5th_closest,6th_closest,7th_closest,8th_closest,9th_closest,10th_closest,11th_closest,12th_closest,13th_closest,14th_closest,15th_closest,16th_closest,17th_closest,18th_closest,19th_closest,20th_closest,21st_closest,22nd_closest,23rd_closest,24th_closest,25th_closest,26th_closest,27th_closest,28th_closest
0,dsgdb9nsd_033805,1,2,3,7.0,9.0,4.0,14.0,16.0,11.0,10.0,6.0,8.0,5.0,13.0,15.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dsgdb9nsd_033805,2,1,3,9.0,7.0,4.0,16.0,14.0,11.0,10.0,8.0,6.0,5.0,15.0,13.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dsgdb9nsd_033805,3,2,9,7.0,4.0,10.0,11.0,5.0,8.0,6.0,16.0,14.0,1.0,12.0,15.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dsgdb9nsd_033805,4,11,10,5.0,3.0,12.0,6.0,8.0,9.0,7.0,2.0,13.0,15.0,16.0,14.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dsgdb9nsd_033805,5,12,6,8.0,4.0,10.0,11.0,3.0,7.0,9.0,13.0,15.0,14.0,16.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Fix bug with the indexes being off by one
if closest['closest'].min() == 1:
    closest['atom_index'] = closest['atom_index'] - 1
    for col in closest.columns:
        if 'close' in col:
            closest[col] = closest[col] - 1

In [15]:
tt = pd.merge(tt,
         closest[['molecule_name','atom_index','closest','2nd_closest','3rd_closest','4th_closest']],
        left_on=['molecule_name','atom_index_0'],
        right_on=['molecule_name','atom_index'],
        how='left') \
    .drop(['atom_index'], axis=1) \
    .rename(columns={'closest':'closest_to_0',
                     '2nd_closest':'2nd_closest_to_0',
                     '3rd_closest':'3rd_closest_to_0',
                     '4th_closest':'4th_closest_to_0'})

In [16]:
tt = pd.merge(tt,
         closest[['molecule_name','atom_index','closest','2nd_closest','3rd_closest','4th_closest']],
        left_on=['molecule_name','atom_index_1'],
        right_on=['molecule_name','atom_index'],
        how='left') \
    .drop(['atom_index'], axis=1) \
    .rename(columns={'closest':'closest_to_1',
                     '2nd_closest':'2nd_closest_to_1',
                     '3rd_closest':'3rd_closest_to_1',
                     '4th_closest':'4th_closest_to_1'})

In [17]:
tt.shape

(7163689, 18)

In [18]:
tt['is_closest_pair'] = tt['closest_to_0'] == tt['atom_index_1']

In [19]:
tt.shape

(7163689, 19)

In [20]:
tt.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,left_middle_average_angle,right_middle_average_angle,distance,is_bond,closest_to_0,2nd_closest_to_0,3rd_closest_to_0,4th_closest_to_0,closest_to_1,2nd_closest_to_1,3rd_closest_to_1,4th_closest_to_1,is_closest_pair
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,NaN,NaN,1.091953,True,0,2,3.0,4.0,3,4,2.0,1.0,True
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,NaN,NaN,1.783120,False,0,2,3.0,4.0,0,1,4.0,3.0,False
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,NaN,NaN,1.783147,False,0,2,3.0,4.0,0,1,4.0,2.0,False
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,NaN,NaN,1.783157,False,0,2,3.0,4.0,0,3,2.0,1.0,False
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,NaN,NaN,1.091952,True,0,1,4.0,3.0,3,4,2.0,1.0,True
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541,NaN,NaN,1.783158,False,0,1,4.0,3.0,0,1,4.0,2.0,False
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548,NaN,NaN,1.783148,False,0,1,4.0,3.0,0,3,2.0,1.0,False
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,NaN,NaN,1.091946,True,0,1,4.0,2.0,3,4,2.0,1.0,True
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543,NaN,NaN,1.783148,False,0,1,4.0,2.0,0,3,2.0,1.0,False
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,NaN,NaN,1.091948,True,0,3,2.0,1.0,3,4,2.0,1.0,True


In [21]:
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt.shape

(7163689, 21)

In [22]:
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','2nd_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_2nd_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt.shape

(7163689, 23)

In [23]:
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','3rd_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_3rd_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt.shape

(7163689, 25)

In [24]:
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','4th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_4th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt.shape

(7163689, 27)

In [25]:
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', 'closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '2nd_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_2nd_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '3rd_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_3rd_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '4th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_4th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt.shape

(7163689, 35)

In [26]:
torsion_details.head()

,molecule_name,2left_atom_idx,left_atom_idx,right_atom_idx,2right_atom_idx,torsion_angle
0,dsgdb9nsd_033805,15,8,7,14,-0.113968
1,dsgdb9nsd_033805,14,7,8,15,-0.113968
2,dsgdb9nsd_033805,15,8,7,4,-176.941197
3,dsgdb9nsd_033805,4,7,8,15,-176.941197
4,dsgdb9nsd_033805,2,8,7,14,176.753908


In [27]:
tt = pd.merge(tt,
        torsion_details.groupby(['molecule_name','2left_atom_idx','left_atom_idx'])['torsion_angle'] \
                  .agg(['mean','min','max','count'])\
                  .reset_index(),
        left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2left_atom_idx','left_atom_idx'],
         how='left') \
    .drop(['2left_atom_idx','left_atom_idx'], axis=1) \
    .rename(columns={'mean': 'tor_ang_2leftleft_mean',
                     'min': 'tor_ang_2leftleft_min',
                    'max': 'tor_ang_2leftleft_max',
                    'count': 'tor_ang_2leftleft_count'})
tt.shape

(7163689, 39)

In [28]:
tt = pd.merge(tt,
         torsion_details.groupby(['molecule_name','2left_atom_idx','right_atom_idx'])['torsion_angle'] \
                  .agg(['mean','min','max','count'])\
                  .reset_index(),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2left_atom_idx','right_atom_idx'],
         how='left') \
    .drop(['2left_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'mean': 'tor_ang_2leftright_mean',
                     'min': 'tor_ang_2leftright_min',
                     'max': 'tor_ang_2leftright_max',
                     'count': 'tor_ang_2leftright_count'})
tt.shape

(7163689, 43)

In [29]:
molecule_details.head()

,molecule_name,mol_wt,num_atoms,num_bonds,num_residues
0,dsgdb9nsd_033805,117.14788,16,17,0
1,dsgdb9nsd_018833,110.15370,18,20,0
2,dsgdb9nsd_013391,115.13046,17,17,0
3,dsgdb9nsd_063427,123.19552,22,22,0
4,dsgdb9nsd_106472,125.16834,20,21,0


In [30]:
tt = pd.merge(tt,
        molecule_details.drop('num_residues', axis=1),
         left_on=['molecule_name'],
         right_on=['molecule_name'],
         how='left')

In [31]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [32]:
test_FE008 = tt.sort_values('id').loc[tt['scalar_coupling_constant'].isnull()].reset_index()
train_FE008 = tt.sort_values('id').loc[~tt['scalar_coupling_constant'].isnull()].reset_index()

In [33]:
test_FE008.shape

(2505542, 47)

In [34]:
test.shape

(2505542, 5)

In [35]:
train_FE008.shape

(4658147, 47)

In [36]:
train.shape

(4658147, 6)

In [53]:
bool_cols = [col for col in train_FE008.columns if 'is_bond_' in col]
test_FE008[bool_cols] = test_FE008[bool_cols].fillna(False)
train_FE008[bool_cols] = train_FE008[bool_cols].fillna(False)

In [54]:
test_FE008.to_parquet('../data/FE008_test.parquet')

In [55]:
train_FE008.to_parquet('../data/FE008_train.parquet')

In [56]:
[col for col in test_FE008.columns]

['index',
 'id',
 'molecule_name',
 'atom_index_0',
 'atom_index_1',
 'type',
 'scalar_coupling_constant',
 'left_middle_average_angle',
 'right_middle_average_angle',
 'distance',
 'is_bond',
 'closest_to_0',
 '2nd_closest_to_0',
 '3rd_closest_to_0',
 '4th_closest_to_0',
 'closest_to_1',
 '2nd_closest_to_1',
 '3rd_closest_to_1',
 '4th_closest_to_1',
 'is_closest_pair',
 'distance_closest_to_0',
 'is_bond_closest_to_0',
 'distance_2nd_closest_to_0',
 'is_bond_2nd_closest_to_0',
 'distance_3rd_closest_to_0',
 'is_bond_3rd_closest_to_0',
 'distance_4th_closest_to_0',
 'is_bond_4th_closest_to_0',
 'distance_closest_to_1',
 'is_bond_closest_to_1',
 'distance_2nd_closest_to_1',
 'is_bond_2nd_closest_to_1',
 'distance_3rd_closest_to_1',
 'is_bond_3rd_closest_to_1',
 'distance_4th_closest_to_1',
 'is_bond_4th_closest_to_1',
 'tor_ang_2leftleft_mean',
 'tor_ang_2leftleft_min',
 'tor_ang_2leftleft_max',
 'tor_ang_2leftleft_count',
 'tor_ang_2leftright_mean',
 'tor_ang_2leftright_min',
 'tor_ang